In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
#%matplotlib tk

In [2]:
result = pd.read_csv("/home/lin/git/summerProject/Heston++onSPXonly_num_100run")

In [3]:
result[['time', 'iteration', 'pv', 'jac', 'lin_sys', 'e0', 'e*', 'Je', 'Dp', 'count','count2', 'count7']].describe()

,time,iteration,pv,jac,lin_sys,e0,e*,Je,Dp,count,count2,count7
count,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000e+02,1.000000e+02,1.000000e+02,100.000000,100.000000,100.0,100.000000
mean,5.388109,761.540000,1790.200000,73.400000,761.600000,2.101206e+05,3.307588e+04,1.040762e+05,0.000083,1.390000,0.0,0.390000
std,20.795142,3094.377271,7054.962857,282.927017,3094.404585,1.351410e+06,3.307588e+05,1.040762e+06,0.000265,0.983654,0.0,0.983654
min,0.183818,22.000000,65.000000,3.000000,22.000000,9.874506e-04,1.110573e-11,4.597017e-17,0.000000,1.000000,0.0,0.000000
25%,0.662529,84.500000,211.500000,9.000000,84.500000,4.302101e-02,8.661107e-11,1.105764e-05,0.000005,1.000000,0.0,0.000000
50%,1.487893,206.500000,501.500000,21.000000,206.500000,1.427714e-01,9.687205e-11,4.959657e-05,0.000023,1.000000,0.0,0.000000
75%,3.067744,424.000000,1058.500000,45.250000,424.000000,2.646230e-01,9.969137e-11,1.653122e-04,0.000086,1.000000,0.0,0.000000
max,200.537157,30000.000000,68193.000000,2728.000000,30000.000000,1.083170e+07,3.307588e+06,1.040762e+07,0.002553,7.000000,0.0,6.000000


In [4]:
result['stopped_by'].unique()
#This show the reasons for stopping:
#1 - stopped by small gradient J^T e
#2 - stopped by small Dp
#3 - stopped by itmax
#4 - singular matrix. Restart from current p with increased \mu
#5 - no further error reduction is possible. Restart with increased mu
#6 - stopped by small ||e||_2
#7 - stopped by invalid (i.e. NaN or Inf) "func" values; a user error

array([6, 5, 1, 3])

In [5]:
#number of solved cases, allowing restart when stopped by 7 or 2
solved = result[result['stopped_by'] == 6]
solved.count().unique()

array([82])

In [6]:
#percentage of solved
solved.count().unique()/result.count().unique()

array([0.82])

In [7]:
#number of solved cases at the first try
solved_first_time = solved[solved['count'] == 1]
solved_first_time.count().unique()

array([70])

In [8]:
#number of cases stopped by small Dp (update of parameter)
DP = result[result['stopped_by'] == 2]
DP.count().unique()

array([0])

In [9]:
#number of cases that stopped by max iteration
itmax = result[result['stopped_by'] == 3]
itmax.count().unique()

array([1])

In [10]:
#number of cases that stopped by mall gradient J^T e
smalJac = result[result['stopped_by'] == 1]
smalJac.count().unique()

array([11])

In [11]:
#average time for all solved cases
solved['time'].describe()['mean']

2.617482292682926

In [12]:
#average time for cases solved without restart
solved_first_time['time'].describe()['mean']

2.6015538142857144

In [13]:
#Average number of iteration for solved cases
solved_first_time['iteration'].describe()['mean']

358.3285714285714

In [14]:
#Average number of iteration for all solved cases
solved['iteration'].describe()['mean']

359.7926829268293

In [15]:
#All solved cases
solved[['Ok', 'Rk', 'Ovbar', 'Rvbar','Ov0', 'Rv0', 'Orho', 'Rrho', 'Osigma', 'Rsigma', 'OI1', 'RI1', 'OI2', 'RI2', 'stopped_by', 'e*']]

,Ok,Rk,Ovbar,Rvbar,Ov0,Rv0,Orho,Rrho,Osigma,Rsigma,OI1,RI1,OI2,RI2,stopped_by,e*
0,1.101482,1.091920,0.460508,0.462785,0.247157,0.247063,-0.642976,-0.643092,0.891423,0.891224,0.000517,0.000519,0.000033,3.457211e-05,6,7.488947e-11
2,4.237238,4.539453,0.067278,0.104508,0.507463,0.504071,-0.355506,-0.355226,0.487375,0.494579,0.000049,0.000091,0.000261,7.374908e-05,6,9.990482e-11
3,2.421126,3.332945,0.339976,0.702871,0.584745,0.849715,-0.444153,-0.345057,0.573649,0.511946,0.011917,0.000846,0.012309,8.415106e-04,6,9.955362e-11
4,2.848193,4.508319,1.311450,0.807836,0.356114,0.241476,-0.174441,-0.204342,0.237646,0.297129,-0.004729,0.000414,-0.005713,7.098196e-04,6,9.950211e-11
7,2.748393,4.444546,0.537655,0.706773,0.933380,0.694078,-0.781358,-0.665228,0.041678,0.067183,-0.007521,0.000525,-0.004900,6.519387e-05,6,9.553267e-11
8,3.795721,2.815968,0.272563,0.445753,0.293362,0.775985,-0.448639,-0.269215,0.303314,0.188244,0.019117,0.000616,0.016196,8.789995e-07,6,6.861742e-11
9,1.259379,1.460961,0.341566,0.399941,1.044059,0.902791,-0.291474,-0.315372,0.283568,0.305632,-0.004772,0.000784,-0.004775,2.821559e-04,6,9.085516e-11
10,0.965031,0.957368,0.615777,0.621246,0.695765,0.765293,-0.771128,-0.702352,0.621721,0.620087,0.003357,0.000598,0.002890,3.187779e-04,6,7.663584e-11
11,1.059134,1.922033,0.137438,0.525693,0.560368,0.579307,-0.465136,-0.444678,0.375836,0.383204,0.001794,0.000447,0.002832,3.878312e-04,6,9.949526e-11
13,2.624883,2.173594,0.326741,0.329152,0.143679,0.143418,-0.705015,-0.720704,0.101855,0.099751,0.000037,0.000160,0.000003,2.970221e-04,6,6.327355e-11


In [16]:
#next 7 blocks show the average distance between initial guess and real parameter
disIk = np.abs(solved['Ik'] - solved['Rk']).mean()
disIk

1.3127838853444715

In [17]:
disIvbar = np.abs(solved['Ivbar'] - solved['Rvbar']).mean()
disIvbar

0.254162999068965

In [18]:
disIv0 = np.abs(solved['Iv0'] - solved['Rv0']).mean()
disIv0

0.26678791367097665

In [19]:
disIsigma = np.abs(solved['Isigma'] - solved['Rsigma']).mean()
disIsigma

0.2843712988409799

In [20]:
disIrho = np.abs(solved['Irho'] - solved['Rrho']).mean()
disIrho

0.22712977540061483

In [21]:
disII1 = np.abs(solved['II1'] - solved['RI1']).mean()
disII1

0.0003184227905100554

In [22]:
disII2 = np.abs(solved['II2'] - solved['RI2']).mean()
disII2

0.0002872592169731074

In [23]:
#next 7 blocks show the average distance between optimal found and real parameter
disOk = np.abs(solved['Ok'] - solved['Rk']).mean()
disOk

0.5330455100751733

In [24]:
disOvbar = np.abs(solved['Ovbar'] - solved['Rvbar']).mean()
disOvbar

0.12365936004381864

In [25]:
disOsigma = np.abs(solved['Osigma'] - solved['Rsigma']).mean()
disOsigma

0.02777336872357968

In [26]:
disOrho = np.abs(solved['Orho'] - solved['Rrho']).mean()
disOrho

0.09216262964701982

In [27]:
disOv0 = np.abs(solved['Ov0'] - solved['Rv0']).mean()
disOv0

0.09062995305593567

In [28]:
disOI1 = np.abs(solved['OI1'] - solved['RI1']).mean()
disOI1

0.003455402948824419

In [29]:
disOI2 = np.abs(solved['OI2'] - solved['RI2']).mean()
disOI2

0.003041995834065906